In [11]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
#import matplotlib.pyplot as plt
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
import pickle
from sklearn.model_selection import train_test_split
import torch.optim as optim

from helper_functions import train_or_load


### Load shadow model and dataset

In [12]:
#load the shadow model trained in the other python script
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # sets to gpu if you have one

mobilenet_shadow =  models.mobilenet_v2(pretrained=False,num_classes=10)
mobilenet_shadow.load_state_dict(torch.load("shadow_models/mobilenet_shadow_cifar_overtrained.pth",map_location=device))

c:\Users\User-1\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\User-1\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


<All keys matched successfully>

In [13]:
DATA_PATH = 'pickle/cifar10/mobilenetv2/shadow.p'
# Change the DATA_PATH to your local pickle file path

with open(DATA_PATH, "rb") as f:
    dataset = pickle.load(f)


#splitting
#only use train set here
train_data, val_data = train_test_split(dataset, test_size=(1-0.5),shuffle=False)
  
train_loader = torch.utils.data.DataLoader(train_data, batch_size=1, shuffle=False, num_workers=2)
testloader =  torch.utils.data.DataLoader(val_data, batch_size=1, shuffle=True, num_workers=2)

for batch_idx, (img, label) in enumerate(train_loader):
    img = img.to(device)


### Generate Attack model Dataset

In [14]:
#generate dataset for attack model
mobilenet_shadow.eval()
dataset_attack = []
# NON Members
with torch.no_grad():
    for images, labels in testloader: #need only one
            # Move images and labels to the appropriate device
        images, labels = images.to(device), labels.to(device)
            # Forward pass
        logits = mobilenet_shadow(images)
        #take the 3 biggest logist
        top_values = torch.topk(logits, k=3).values
        top_values, indices = torch.sort(top_values, dim=1, descending=True)
        dataset_attack.append([top_values,0])
# MEMBERS
with torch.no_grad():
    for images, labels in train_loader: #need only one
            # Move images and labels to the appropriate device
        images, labels = images.to(device), labels.to(device)
            # Forward pass
        logits = mobilenet_shadow(images)
        #take the 3 biggest logist
        top_values = torch.topk(logits, k=3).values
        top_values, indices = torch.sort(top_values, dim=1, descending=True)
        dataset_attack.append([top_values,1])
        



In [15]:
standardized_data_list = dataset_attack

# # Convert all tensors to the same dtype first
# tensors = [data[0].float() for data in dataset_attack]  # Ensure all tensors are Float type
# all_data = torch.cat(tensors, dim=0)  # Concatenate all tensors

# # Calculate mean and std
# mean = all_data.mean(dim=0)
# std = all_data.std(dim=0)

# # Standardize data in the list
# standardized_data_list = [( (data[0] - mean) / std, data[1] ) for data in dataset_attack]


'\n# Convert all tensors to the same dtype first\ntensors = [data[0].float() for data in dataset_attack]  # Ensure all tensors are Float type\nall_data = torch.cat(tensors, dim=0)  # Concatenate all tensors\n\n# Calculate mean and std\nmean = all_data.mean(dim=0)\nstd = all_data.std(dim=0)\n\n# Standardize data in the list\nstandardized_data_list = [( (data[0] - mean) / std, data[1] ) for data in dataset_attack]\n'

In [16]:
attack_dataloader = torch.utils.data.DataLoader(standardized_data_list, batch_size=64, shuffle=True, num_workers=2) #shuffled training data

In [29]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(3, 32)
        self.bn1 = nn.BatchNorm1d(32)
        self.fc2 = nn.Linear(32, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.fc3 = nn.Linear(64, 32)
        self.bn3 = nn.BatchNorm1d(32)
        self.fc4 = nn.Linear(32, 16)
        self.bn4 = nn.BatchNorm1d(16)
        self.fc5 = nn.Linear(16, 8)
        self.bn5 = nn.BatchNorm1d(8)
        self.fc6 = nn.Linear(8, 1)
        self.dropout = nn.Dropout(0.5)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))
        x = torch.relu(self.bn2(self.fc2(x)))
        x = torch.relu(self.bn3(self.fc3(x)))
        x = torch.relu(self.bn4(self.fc4(x)))
        x = self.dropout(torch.relu(self.bn5(self.fc5(x))))
        x = self.sigmoid(self.fc6(x))
        return x
    


In [ ]:
attack_model = SimpleNN()
criterion = nn.BCELoss()
optimizer = optim.Adam(attack_model.parameters(), lr=0.0001)
attack_model.train()

In [19]:
# for inputs in attack_dataloader:
#     inputs = inputs[0]
#     print(inputs.shape)  # Check input shape consistenc
#     print(inputs)
#     print(inputs)
#     outputs = attack_model(inputs.squeeze(dim = 1))

In [30]:
train_or_load(attack_model,attack_dataloader,optimizer,criterion,epochs=5)

Epoch: 1


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x000001743FBD1DA0>
Traceback (most recent call last):
  File "c:\Users\User-1\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "c:\Users\User-1\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py", line 1436, in _shutdown_workers
    if self._persistent_workers or self._workers_status[worker_id]:
                                   ^^^^^^^^^^^^^^^^^^^^
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_workers_status'


tensor([[0.7569, 0.5706, 0.4115]])
tensor([0.])
Batch 0 with Loss: 0.6691667437553406
tensor([[0.3281, 0.2959, 0.2014]])
tensor([0.])
Batch 30 with Loss: 0.7166800498962402
tensor([[0.4421, 0.2805, 0.2435]])
tensor([1.])
Batch 60 with Loss: 0.6952509880065918
tensor([[0.6007, 0.5285, 0.3627]])
tensor([1.])
Batch 90 with Loss: 0.7080126404762268
tensor([[1.4218, 1.2167, 0.9277]])
tensor([1.])
Batch 120 with Loss: 0.7215996980667114
tensor([[0.5054, 0.4674, 0.3625]])
tensor([0.])
Batch 150 with Loss: 0.6880431175231934
tensor([[0.5406, 0.1882, 0.1676]])
tensor([0.])
Batch 180 with Loss: 0.7156131267547607
tensor([[0.3323, 0.2286, 0.2144]])
tensor([1.])
Batch 210 with Loss: 0.700903594493866
tensor([[0.3211, 0.1837, 0.0598]])
tensor([0.])
Batch 240 with Loss: 0.6900610327720642
tensor([[0.4039, 0.3599, 0.3435]])
tensor([0.])
Batch 270 with Loss: 0.6937681436538696
tensor([[0.2097, 0.1341, 0.1272]])
tensor([1.])
Batch 300 with Loss: 0.6926522850990295
tensor([[0.5366, 0.4885, 0.4006]])
ten

In [31]:
torch.save(attack_model.state_dict(), 'attack_models/attack_mobilenet_cifar.pth')

<h1>Evaluation</h1>

In [32]:
DATA_PATH = "pickle/cifar10/mobilenetv2/eval.p"

device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

with open(DATA_PATH, "rb") as f:
    dataset = pickle.load(f)
# Convert all tensors to the same dtype first

eval_data_loader = torch.utils.data.DataLoader(dataset, batch_size=1 , shuffle=False, num_workers=2)
#splitting
for batch_idx, (img, label, membership) in enumerate(eval_data_loader):
    img = img.to(device)

In [33]:
resnet_target_model =  models.mobilenet_v2(weights=None,num_classes=10)
check = torch.load("models/mobilenetv2_cifar10.pth", map_location=device)
# check["net"]["classifier.0.weight"]=check["net"].pop("classifier.1.weight")
# check["net"]["classifier.0.bias"]=check["net"].pop("classifier.1.bias")

resnet_target_model.load_state_dict(check["net"])
resnet_target_model.eval()

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [40]:
#get posteriors from target
dataset_eval = []
with torch.no_grad():
    for images,_, member in eval_data_loader: # take only image for query and member status (ignore cifar label)
            # Move images and labels to the appropriate device
        images, labels = images.to(device), labels.to(device)
            
            # Forward pass
        logits = resnet_target_model(images)
        
        #take the 3 biggest logist
        top_values = torch.topk(logits, k=3).values #order poseri
        sorted_tensor, indices = torch.sort(top_values, dim=1,descending=True)
        dataset_eval.append([sorted_tensor, member.item()])
        


In [42]:
dataset = dataset_eval
# # Convert all tensors to the same dtype first
# tensors = [data[0].float() for data in dataset_eval]  # Ensure all tensors are Float type
# all_data = torch.cat(tensors, dim=0)  # Concatenate all tensors

# # Calculate mean and std
# mean = all_data.mean(dim=0)
# std = all_data.std(dim=0)

# # Standardize data in the list
# dataset = [( (data[0] - mean) / std, data[1] ) for data in dataset_eval]

In [36]:
dataloader_eval = torch.utils.data.DataLoader(dataset, batch_size=1 , shuffle=False, num_workers=2)

In [41]:
def evaluate_attack_model(model, train_loader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient computation
        for sorted_logits, members in train_loader:
            sorted_logits = sorted_logits.float()
            #labels = torch.tensor(labels[0])
            optimizer.zero_grad()

            outputs = model(sorted_logits.squeeze(dim=1))
            predicted = torch.round(outputs)  # Round the outputs to 0 or 1
            total += members.size(0)  # Increment the total count by batch size
            correct += (predicted == members).sum().item()  # Count correct predictions

    accuracy = correct / total
    return accuracy


accuracy = evaluate_attack_model(attack_model, dataloader_eval)#


print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.71
